# 🎯 PhobiaShield - Training Clean
**The Architect Module - Production Ready**

**Setup:** Runtime > GPU (T4)

---

## 📋 Steps:
1. Setup & GPU check
2. Mount dataset from Google Drive
3. Train model (10 epochs)
4. Evaluate results

## 🔧 Step 1: Setup

In [ ]:
# Clone repo and install
!git clone https://github.com/Gabriele-mp/PhobiaShield.git
%cd PhobiaShield
!git checkout TheArchitect
!pip install -q -e .

print("✅ Setup complete!")

In [ ]:
# GPU Check
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory/1e9:.1f} GB")
    print("✅ GPU ready!")
else:
    print("⚠️  Running on CPU (slow)")

## 📦 Step 2: Load Dataset

**Prerequisites:**
1. Compress dataset locally: `zip -r phobiashield_final.zip data/phobiashield_final/`
2. Upload to Google Drive: `MyDrive/PhobiaShield/phobiashield_final.zip`

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Unzip dataset
!unzip -q /content/drive/MyDrive/PhobiaShield/phobiashield_final.zip -d /content/PhobiaShield/

# Verify
!echo "Train images:"
!ls data/phobiashield_final/train/images | wc -l
!echo "Val images:"
!ls data/phobiashield_final/val/images | wc -l

print("\n✅ Dataset loaded!")

## 🚀 Step 3: Training

Training parameters:
- Epochs: 10
- Batch size: 8
- Learning rate: 0.0001
- Classes: 5 (Clown, Shark, Spider, Blood, Needle)

In [ ]:
# Train model
!python scripts/train_clean.py \
    --data data/phobiashield_final \
    --config cfg/model/tiny_yolo_5class.yaml \
    --epochs 10 \
    --batch-size 8 \
    --lr 0.0001 \
    --device cuda

## 📊 Step 4: Evaluate Results

In [ ]:
# Load trained model
from omegaconf import OmegaConf
from src.models.phobia_net import PhobiaNet
import matplotlib.pyplot as plt
from pathlib import Path

# Load config and model
model_cfg = OmegaConf.load('cfg/model/tiny_yolo_5class.yaml')
model = PhobiaNet(model_cfg).to(device)

# Load best weights
checkpoint = torch.load('outputs/checkpoints/best_model.pth')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print(f"✅ Model loaded!")
print(f"   Epoch: {checkpoint['epoch']+1}")
print(f"   Train Loss: {checkpoint['train_loss']:.2f}")
print(f"   Val Loss: {checkpoint['val_loss']:.2f}")

In [ ]:
# Test on validation images
from PIL import Image
import numpy as np
import torchvision.transforms as T

# Get random val images
val_images = list(Path('data/phobiashield_final/val/images').glob('*.jpg'))[:6]

# Transform
transform = T.Compose([
    T.Resize((416, 416)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Class names
class_names = ['Clown', 'Shark', 'Spider', 'Blood', 'Needle']

# Plot
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, img_path in enumerate(val_images):
    # Load and transform
    img = Image.open(img_path).convert('RGB')
    img_tensor = transform(img).unsqueeze(0).to(device)
    
    # Inference
    with torch.no_grad():
        output = model(img_tensor)
    
    # Display
    axes[idx].imshow(img)
    axes[idx].axis('off')
    axes[idx].set_title(f"{img_path.name}")

plt.tight_layout()
plt.show()

print("✅ Inference complete!")
print("\nNote: Full detection with NMS requires additional implementation.")
print("Output shape:", output.shape, "(batch, grid_h, grid_w, predictions)")

## 📈 Training Summary

**Expected Results (10 epochs):**
- Train Loss: ~50-80
- Val Loss: ~60-90
- Training time: ~15-20 min on T4

**Next Steps:**
1. Increase epochs to 50-100 for better performance
2. Implement NMS for full detection pipeline
3. Calculate mAP on test set
4. Deploy to inference pipeline